In [3]:
from Helper.Load.load import Base_Load
from Helper.Load.supervised import Supervised
from Helper.Load.regression import Regression
from Helper.Load.construct_transform_config import construct_transform_config
from Helper.Load.classification import Classification
from Helper.Visual.Visual import Visual
from Helper.Visual.classification import VClassification
from Helper.Transform.classification import Classificaton_Transform

import numpy as n
import matplotlib.pyplot as plt

In [4]:
#ctc = construct_transform_config("Data", "haberman.dat", "haberman-names.txt")
loadC = Classification("Data", "haberman.dat", "haberman-names.txt")
transformC = Classificaton_Transform(loadC)

In [5]:
loadC.read_transform_config()

{'attribute': {'Age': {'name': 'Age',
   'abstraction': 'Ordinal',
   'selector': 'Number',
   'bounds': None,
   'func': None,
   'params': None},
  'Year': {'name': 'Year',
   'abstraction': 'Ordinal',
   'selector': 'Number',
   'bounds': None,
   'func': None,
   'params': None},
  'Positive': {'name': 'Positive',
   'abstraction': 'Ordinal',
   'selector': 'Number',
   'bounds': None,
   'func': None,
   'params': None},
  'Survival': {'name': 'Survival',
   'abstraction': 'Dichotomous',
   'selector': 'Text',
   'bounds': None,
   'func': None,
   'params': None}},
 'inputs': ['Age', 'Year', 'Positive'],
 'outputs': ['Survival']}

In [6]:
loadC.partition.X_train.head()

,Age,Year,Positive
16,55,66,18
126,57,62,0
152,50,65,4
289,45,68,0
67,50,64,0


In [7]:
transformC.assemble_pipelines().fit_transform(loadC.partition.X_train)[:5, :]

{'name': 'Age', 'abstraction': 'Ordinal', 'selector': 'Number', 'bounds': None, 'func': None, 'params': None}
{'name': 'Year', 'abstraction': 'Ordinal', 'selector': 'Number', 'bounds': None, 'func': None, 'params': None}
{'name': 'Positive', 'abstraction': 'Ordinal', 'selector': 'Number', 'bounds': None, 'func': None, 'params': None}


array([[22.,  8., 17.],
       [24.,  4.,  0.],
       [17.,  7.,  4.],
       [12., 10.,  0.],
       [17.,  6.,  0.]])

In [8]:
pipeline = transformC.assemble_pipelines().fit(loadC.partition.X_train)

{'name': 'Age', 'abstraction': 'Ordinal', 'selector': 'Number', 'bounds': None, 'func': None, 'params': None}
{'name': 'Year', 'abstraction': 'Ordinal', 'selector': 'Number', 'bounds': None, 'func': None, 'params': None}
{'name': 'Positive', 'abstraction': 'Ordinal', 'selector': 'Number', 'bounds': None, 'func': None, 'params': None}


In [9]:
pipeline.transform(loadC.partition.X_test)

ValueError: Found unknown categories [33, 40, 71, 75, 76, 83] in column 0 during transform